### The "Adds up to 100" puzzle:
- The question is more or les stated, "Given some vector of "numbers" $X$ of positive numbers, which might be quite large, how many pairs of numbers add up to 100?"
- First, answer the question with a question: What kind of numbers? Integers? Over what range -- are all of the numbers $x_j < 100$?
- Here, we will provide reasonably general solutions for both the continuum and integer cases.
- Note that fundamentally, this is an $ON^2$ problem, but we want to reduce that as much as possible.e 

#### Summary findings:
- We can use sorting based strategies to significantly reduce the order of the algorithm for continuum sequences, though the performance of such an algorithm might be difficult to anticipate
- For integers, we can reduce the problem to approximately $ON$, or $c \to ON$ in the limit of very large $N$. This reduction arises from the simple fact that, no matter how large $N$ is, there are only 101 distinct values of interest, all positive integers $x \le 100$, including $0$. Therefore, we can index the set like, $I = \{ x:n_x, ... \}$ where $0 \le x \le 100$ are the values of $x$ and $n_x$ are the number of that value in the set.
- We can now look for the number of pairs in this reduced set, and in fact, because the valuse are integer type, we can again use our index. For each value $x_j$, we can compute its compliment, $x^c_j = 100 - x_j$; we can then look up $x^c_j$ in our index, and the

In [18]:
%load_ext autoreload
%autoreload 2
%matplotlib inline



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
import matplotlib
import pylab as plt
import numpy
import scipy
import random


### Make a sequence of numbers

In [20]:
N = 1000
R = random.Random()

x_min = 0.
x_max = 100.

# search range:
x_lo = 50.
x_hi = 60.

X = [R.random()*(x_min + (x_max - x_min)) for _ in range(N)]


### Continuum case:
- First, we might assume based on how the question was phrased that we can ignore this case. Continuum values never really equal oneanother, so we'll modify the question to say something like, find the pairs of numbers whose sum is between $x_{min} < x_1 + x_2 < x_{max}$.
- We don't have too much to work with; this problem is still fundamentally (as much as) $ON^2$, but:
    - We can use sorting to reduce the scope a bit
    - Obviuosly, we can exclude any numbers $x>x_{max}$
    - ... etc.
- So this gets us approximately $O(N \cdot \log(N) + \alpha N^2)$, where $\alpha$ is more or less (exactly?) the square of the fraction of values $x < x_{max}$.
- So we observe significant reductions in the complexity, even for $x_{hi} \approx x_{max}$. The next step might be to figure out a witty indexing or modulus-skipping system to optimize the search for small $x$.


In [21]:
# now, find all pairs where x1 < (x_j + x_k) < x2
#
# make a sorted copy of X:
X_prime = list(sorted(X))
pairs = []
nits = 0
#
# we should maybe (or maybe not) use itertools, but first, let's just block it out:
for j, x1 in enumerate(X_prime):
    # is this a valid value for x1? if we've already exceeded our maximum value, we can stop spinning...
    if x1>x_hi: break
    #
    for k, x2 in enumerate(X_prime):
        nits += 1
        if x2 + x1 > x_hi:
            break
        x = x1 + x2
        if x>=x_lo and x<=x_hi:
            pairs += [[j,k, x1, x2, x]]
        #
#
#       

print("Number of distinct pairs: {}".format(len(pairs)))
print('nits: {}/{} ({:.3f})'.format(nits, N*N, nits/(N*N)))

Number of distinct pairs: 53695
nits: 169074/1000000 (0.169)


#### Variable Types: The Integer trick

- So that's the sort of problem we would probably encounter in physical science -- continuum values.
- But in CS, or just for the sake of puzzles, let's consider the case where $X$ is restricted to positive ingegers, this problem becomes approximately $ON$ for large $N$. The number of integers that add to 100 is small (it's 100).
- We solve the problem in two steps. We index and filter the input:
    - Exclude all values $x>100$; count and index the number of each value $x \le 100$
    - Now, for each entry $x_j$, compute the colplement $x_k = 100 - x_j$, and add the total count, $n = n_j \cdot n_k$.
    - Note that we return the pairs, $x_j + x_k = 100$, and their counts. These are $2 \times$ degenerate, in that we get two redundant pairs, $(x_j, x_k) = (x_k, x_j)$, so if we want the total count, add these up and divide by two. 
    - ...or equivalently, add up half of them. So we only need to spin over $0 < x_1 < 50$.
- Note also that, we interpret this challenge with the simplified rule that an element can be use multiple times, in multiple pairs. If we can only use an entry once, the number of single-use pairs is $n=min(n_j, n_k)$.

In [46]:
N = 10000
R = random.Random()
#
x_min = 0.
x_max = 200.
#
X = [int(R.random()*(x_min + (x_max - x_min))) for _ in range(N)]
#
X_index = {x:0 for x in range(int(x_min), 101, 1) }
for x in X:
    if x>100: 
        continue
    #
    X_index[x] +=1
#
pairs = []
#
for x,n in X_index.items():
    # we might sort, or we can just skip x>x_0 values...
    if x>50: continue
    #if x>100: continue
    x_prime = int(100 - x)
    n_prime = X_index[x_prime]
    #
    pairs += [[(x, x_prime), (n, n_prime), n*n_prime]]
# ... but we will still count 50 twice.note that 50+50 should occur approximately half as often
# as other combinations, since it is the same one value repeated, not two values.
# generally, this might be considered an example of the kind of erro we might incur by trying to
# over optimize -- introducing complexities to the code that lend themselves to mistakes, but don't
# really add much benefit (how much do we really need the O(N/2) boost? at what cost in terms of possible
# errors?)
# now
pairs[50][-1]/=2
n_pairs = numpy.sum([rw[-1] for rw in pairs])
#
#
# a quick idot-check:
print('n[100] = {}, n[3]={}, n[5]={}'.format(X.count(100), X.count(3), X.count(5)))
print('total pairs: {}'.format(n_pairs))
#
print('pairs: ')
for p in pairs: print(p)


n[100] = 45, n[3]=49, n[5]=46
total pairs: 126673.0
pairs: 
[(0, 100), (45, 45), 2025]
[(1, 99), (50, 43), 2150]
[(2, 98), (40, 62), 2480]
[(3, 97), (49, 56), 2744]
[(4, 96), (53, 51), 2703]
[(5, 95), (46, 40), 1840]
[(6, 94), (62, 49), 3038]
[(7, 93), (46, 57), 2622]
[(8, 92), (40, 44), 1760]
[(9, 91), (46, 50), 2300]
[(10, 90), (52, 53), 2756]
[(11, 89), (47, 46), 2162]
[(12, 88), (52, 48), 2496]
[(13, 87), (51, 47), 2397]
[(14, 86), (54, 53), 2862]
[(15, 85), (49, 39), 1911]
[(16, 84), (47, 51), 2397]
[(17, 83), (49, 46), 2254]
[(18, 82), (43, 58), 2494]
[(19, 81), (53, 51), 2703]
[(20, 80), (60, 48), 2880]
[(21, 79), (55, 58), 3190]
[(22, 78), (52, 50), 2600]
[(23, 77), (54, 38), 2052]
[(24, 76), (51, 47), 2397]
[(25, 75), (45, 55), 2475]
[(26, 74), (65, 55), 3575]
[(27, 73), (48, 46), 2208]
[(28, 72), (50, 58), 2900]
[(29, 71), (61, 37), 2257]
[(30, 70), (43, 47), 2021]
[(31, 69), (58, 48), 2784]
[(32, 68), (54, 44), 2376]
[(33, 67), (67, 63), 4221]
[(34, 66), (56, 58), 3248]
[(35

#### Check results with brute-force method:

In [47]:
import itertools

class Brute_Force_Pair_Counter(list):
    # brute force pair-counter. assume integer inputs, or at least that we can use
    # a == b equivalence. for float values, we really need to write an almost_equal(a,b)
    # operator, but we'll save that for a real application...
    #
    # regarding list[] inheritance... or maybe dict. eventually populate the pairs... mabye use a nested
    # dict{} structure, {x_j:{x_k:n, ...}, }
    def __init__(self, X, target_value=100):
        n_pairs = 0
        for x1,x2 in itertools.product(X,X):
            if x1 + x2 == 100: n_pairs += 1
            #
        #
        self.n_total = n_pairs
    #
BF_pairs = Brute_Force_Pair_Counter(X,100)

In [48]:
print('total: {}'.format(BF_pairs.n_total))
print(n_pairs*2)

total: 253346
253346.0


### Comments: 
- Note that we can hybridize these techniques.
- If we have inherently continuum value data, can we approximate (round or truncate) those values, so that we can index them?
    - $x = 5.1$, $dx = .1$, $j=51$
    - $j_k = int \left( \frac{x_k}{dx} \right)$ 
- It gets a little bit trickeier when we then evaluate a range of equalities, using our indexed system, but it's not too bad; we simply replace our $x_c = 100 - x$ with a framework where we evaluate all $(x_{lo}-x) < x_c < (x_{hi}-x)$. How exactly we do this may depend on the properties of the data.
